In [ ]:
!pip install pymysql
import pymysql
import boto3
import json
import pandas as pd
from IPython.display import display, HTML

def get_secret_name_from_ssm(param_name):
    """retrive secret name from ssm parameter"""
    # create ssm client
    client = boto3.client('ssm')

    # get db parameter
    secret_name = client.get_parameter(Name=param_name)['Parameter']['Value']

    return secret_name

def get_secret(secret_name, region_name):
    """Retrieve RDS database credentials from AWS Secrets Manager."""
    # Create a Secrets Manager client
    client = boto3.client('secretsmanager', region_name=region_name)

    try:
        # Get the secret value from Secrets Manager
        response = client.get_secret_value(SecretId=secret_name)

        # Secrets Manager returns the secret as a JSON string, parse it
        secret = json.loads(response['SecretString'])
        return secret

    except Exception as e:
        print(f"Error retrieving secret: {e}")
        raise

def get_rds_endpoint(rds_instance_identifier):
    """Retrieve RDS instance endpoint from AWS RDS."""
    rds_client = boto3.client('rds')
    response = rds_client.describe_db_clusters(
        DBClusterIdentifier=rds_instance_identifier
    )
    endpoint = response['DBClusters'][0]['Endpoint']
    return endpoint

def read_tables_from_database(rds_instance_identifier, db_name, username, password):
    """Connect to the RDS MySQL database and all the tables from the database."""
    # Get the RDS instance endpoint
    rds_client = boto3.client('rds')
    endpoint = rds_client.describe_db_clusters(
    DBClusterIdentifier=rds_instance_identifier,
    )['DBClusters'][0]['Endpoint']

    # Connect to the RDS MySQL instance
    connection = pymysql.connect(
        host=endpoint,
        user=username,
        password=password,
        database=db_name,
        port=3306,
        cursorclass=pymysql.cursors.DictCursor  # Return rows as dictionaries
    )

    try:
        with connection.cursor() as cursor:
            # Create SQL query to select data
            sql_query = f"SHOW TABLES;"
            cursor.execute(sql_query)
            # Fetch all rows from the table
            result = cursor.fetchall()
            # Get column names from the cursor description
            columns = [desc[0] for desc in cursor.description]

            # Create a pandas DataFrame with the results and column names
            df = pd.DataFrame(result, columns=columns) #, columns=columns)
            return df
    finally:
        connection.close()

# Usage example
rds_instance_identifier = 'survaasdefault-db'
db_name = 'SurvaasDefaultDb'
ssm_param_name = 'SurvaasDefaultDbSecretArn'
region = boto3.session.Session().region_name
username = get_secret(get_secret_name_from_ssm(ssm_param_name), region)["username"]
password = get_secret(get_secret_name_from_ssm(ssm_param_name), region)["password"]

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    tables = read_tables_from_database(rds_instance_identifier, db_name, username, password)
    display(tables)